# Title

# Description

# Importation

In [1]:
# pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

import matplotlib.dates as mdates
%matplotlib inline
from itertools import product

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from category_encoders.binary import BinaryEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error

import warnings
import os
warnings.filterwarnings("ignore")
# from google.colab import drive

# Data Loading

In [3]:
# # Mount Google Drive
# drive.mount('/content/drive')

# # Define the folder path in Google Drive where your CSV files are located
# folder_path = "/content/drive/MyDrive/Colab Notebooks/datasets/grocery store azubian"

# # Load the CSV files into DataFrames
# data = {}

# # Iterate over the files in the folder
# for file_name in os.listdir(folder_path):
#     if file_name.endswith(".csv"):
#         # Remove the file extension to get the variable name
#         variable_name = file_name.replace(".csv", "")
        
#         # Construct the file path
#         file_path = os.path.join(folder_path, file_name)
        
#         # Read the CSV file content into a DataFrame
#         data[variable_name] = pd.read_csv(file_path)

# # Access the data using dictionary keys
# holidays = data["holidays"]
# dates = data["dates"]
# sample = data["SampleSubmission"]
# stores = data["stores"]
# test = data["test"]
# train = data["train"]



In [4]:
train = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/train.csv")
test = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/test.csv")
stores = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/stores.csv")
sample = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/SampleSubmission.csv")
dates = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/dates.csv")
holidays = pd.read_csv("C:/Users/LENOVO/Music/Grocery-Store-Forecasting-Challenge-For-Azubian/assets/grocery store azubian/holidays.csv")

# Dataset overview

In [5]:
train.head()

,date,store_id,category_id,target,onpromotion,nbr_of_transactions
0,365,store_1,category_24,0.0,0,0.0
1,365,store_1,category_21,0.0,0,0.0
2,365,store_1,category_32,0.0,0,0.0
3,365,store_1,category_18,0.0,0,0.0
4,365,store_1,category_26,0.0,0,0.0


train date is in numerical format. We'll have to convert it to Datetime format later

In [6]:
test.head()

,date,store_id,category_id,onpromotion
0,1627,store_1,category_24,0
1,1627,store_1,category_21,0
2,1627,store_1,category_32,0
3,1627,store_1,category_18,16
4,1627,store_1,category_26,0


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99792 entries, 0 to 99791
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         99792 non-null  int64 
 1   store_id     99792 non-null  object
 2   category_id  99792 non-null  object
 3   onpromotion  99792 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 3.0+ MB


In [8]:
stores.head()

,store_id,city,type,cluster
0,store_1,0,0,0
1,store_2,0,0,0
2,store_3,0,0,1
3,store_4,0,0,2
4,store_5,1,0,3


In [9]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   store_id  54 non-null     object
 1   city      54 non-null     int64 
 2   type      54 non-null     int64 
 3   cluster   54 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.8+ KB


city, type & cluster are categoric variables, so they are not supposed to be in number datatype(int64) 

furthermore, cities do not have an ordinal relationship with one another. Ordinal variables have a natural order. Just like "good-better-best" or "positive-neutral-negative". Nominal variables don't.

We don't want our machine learning models to think that one city-0 comes before city-1, which is before city-3. 

Therefore, we'll have to change the datatypes to obect or string to make it more descriptive, for example: 'London', 'Tokyo', 'Rome' and so on.

Same goes for type and cluster.

In [10]:
dates['date'].unique()

array([ 365,  366,  367, ..., 1682, 1683, 1684], dtype=int64)

This dataset contains dates and the features that have already been extracted from it

In [11]:
dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   date              1320 non-null   int64
 1   year              1320 non-null   int64
 2   month             1320 non-null   int64
 3   dayofmonth        1320 non-null   int64
 4   dayofweek         1320 non-null   int64
 5   dayofyear         1320 non-null   int64
 6   weekofyear        1320 non-null   int64
 7   quarter           1320 non-null   int64
 8   is_month_start    1320 non-null   bool 
 9   is_month_end      1320 non-null   bool 
 10  is_quarter_start  1320 non-null   bool 
 11  is_quarter_end    1320 non-null   bool 
 12  is_year_start     1320 non-null   bool 
 13  is_year_end       1320 non-null   bool 
 14  year_weekofyear   1320 non-null   int64
dtypes: bool(6), int64(9)
memory usage: 100.7 KB


in this case, these categories have an ordinal relationship with one another, meaning one date naturally comes before the other, 

so we can leave them as they are.

In [12]:
dates.dayofyear.unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
       118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
       157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
       170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 18

we have 365 days in a year, 366 days is for a loop year. This is a problem for us. Let me explain why.

**Problem**:
When you have a loop year, then new year's eve would fall on day 366

otherwise, it would fall on day 365. So everyday might not fall on the appropriate number for each year.

**Solution**:
we will later convert two new columns called "sin(dayofyear)" & "cos(dayofyear)". These new columns will help our machine learning models to understand the cyclic nature of a year. 

Cyclic means that a year usually starts and ends in a similar way.

In [13]:
holidays.head()

,date,type
0,1,0
1,5,4
2,12,4
3,42,0
4,43,0


In [14]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   date    281 non-null    int64
 1   type    281 non-null    int64
dtypes: int64(2)
memory usage: 4.5 KB


In [15]:
holidays.type.unique()

array([0, 4, 2, 5, 1, 3], dtype=int64)

The type column is a categoric variable, and each type of holiday does not have an ordinal relationship, since a holiday like new year is not higher or better than Christmas for example.

so we'll later convert them to string type to make it more descriptive

In [16]:
train.describe()

,date,target,onpromotion,nbr_of_transactions
count,2.248884e+06,2.248884e+06,2.248884e+06,2.248884e+06
mean,9.955000e+02,3.932649e+02,3.142002e+00,1.573591e+03
std,3.643080e+02,1.174938e+03,1.343616e+01,1.027832e+03
min,3.650000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,6.800000e+02,0.000000e+00,0.000000e+00,9.440000e+02
50%,9.955000e+02,1.500000e+01,0.000000e+00,1.338000e+03
75%,1.311000e+03,2.230000e+02,1.000000e+00,1.979000e+03
max,1.626000e+03,1.247170e+05,7.410000e+02,8.359000e+03


train dates range from **365** to **1626**

In [17]:
test.describe()

,date,onpromotion
count,99792.000000,99792.000000
mean,1654.500000,7.274892
std,16.163311,18.504041
min,1627.000000,0.000000
25%,1640.750000,0.000000
50%,1654.500000,0.000000
75%,1668.250000,6.000000
max,1682.000000,591.000000


test dates range from **1627** to **1682**

this is a continuation from train. This makes sense since we are to predict future transactions based on past data

**note**: we will not be using transaction data to train our models, since transaction data was not provided for our test data.

In [18]:
dates.describe(),

(              date         year        month   dayofmonth    dayofweek  \
 count  1320.000000  1320.000000  1320.000000  1320.000000  1320.000000   
 mean   1024.500000     2.344697     6.124242    15.612121     2.996970   
 std     381.195488     1.059333     3.366835     8.798020     2.001135   
 min     365.000000     1.000000     1.000000     1.000000     0.000000   
 25%     694.750000     1.000000     3.000000     8.000000     1.000000   
 50%    1024.500000     2.000000     6.000000    15.500000     3.000000   
 75%    1354.250000     3.000000     9.000000    23.000000     5.000000   
 max    1684.000000     4.000000    12.000000    31.000000     6.000000   
 
          dayofyear   weekofyear      quarter  year_weekofyear  
 count  1320.000000  1320.000000  1320.000000      1320.000000  
 mean    170.871970    24.934848     2.384091       259.101515  
 std     102.870939    14.726066     1.094568       104.162039  
 min       1.000000     1.000000     1.000000       101.000000 

dates are from **365** till **1684** which covers the train and test dates

so, we'll be able to add the features from here to both the train and test data based on the date

In [19]:
 holidays.describe()

,date,type
count,281.000000,281.000000
mean,901.896797,1.402135
std,461.359327,1.996031
min,1.000000,0.000000
25%,540.000000,0.000000
50%,934.000000,0.000000
75%,1237.000000,2.000000
max,1684.000000,5.000000


In [20]:
# count the number of dates in the holidays dataset
holidays.date.nunique()

251

notice that the dates in the holiday dataset are not complete

so, we will later create a column for holidays in our train and test dataset based on the following logic:

if a date is in the holidays table, then its a holiday, else that date is not a holiday

## Hypothesis
**H0**: holidays have a big effect on sales, hence the sales data is seasonal.

**H1**: holidays don't affect sales, hence sales data is stationary.

## Questions

1. Is the train data complete?
2. Do we have seasonality in our sales?
3. Are there outliers in our dataset?
4. What is the difference between RMSLE, RMSE and MSE?

| Issues                                  | how we intend to solve them                                                                                                   |
|----------------------------------------|------------------------------------------------------------------------------------------------------------|
| 1. City, type & cluster in our stores dataset are mumerical | convert to string and make the categories more descriptive.                                            |
| 2. The dayofyear column in our dates dataset ranges from 1 to 366. This will make some days fall on the wrong number | find the sine and cosine of this column to represent the cyclic nature of a year. | We can also include weather conditions, holidays and events to this.                        |


# Data Cleaning

Here, we will prepare our data for Univariate and Bivariate analysis. 

## Fixing our issues

1. City, type & cluster in our stores dataset are mumerical 

Solution: convert to string and make the categories more descriptive.

**city**

In [21]:
stores.city.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21], dtype=int64)

In [22]:
# using each city number as index, 
# convert each city number to the corresponding city from a list of us_cities
stores.city = stores.city.apply(lambda x: 'city_'+ str(x))

In [23]:
stores.city.unique()

array(['city_0', 'city_1', 'city_2', 'city_3', 'city_4', 'city_5',
       'city_6', 'city_7', 'city_8', 'city_9', 'city_10', 'city_11',
       'city_12', 'city_13', 'city_14', 'city_15', 'city_16', 'city_17',
       'city_18', 'city_19', 'city_20', 'city_21'], dtype=object)

**type**

In [24]:
stores.type.unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [25]:
# convert each store_type number to the corresponding store_type from a list of grocery_store_types
stores.type = stores.type.apply(lambda x: 'store_'+ str(x))

In [26]:
stores.type.unique()

array(['store_0', 'store_1', 'store_2', 'store_3', 'store_4'],
      dtype=object)

**cluster**

In [27]:
stores.cluster.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16],
      dtype=int64)

In [28]:
# convert each cluster number to the corresponding cluster from a list of us_cities
stores.cluster = stores.cluster.apply(lambda x: 'cluster_'+ str(x))

In [29]:
stores.cluster.unique()

array(['cluster_0', 'cluster_1', 'cluster_2', 'cluster_3', 'cluster_4',
       'cluster_5', 'cluster_6', 'cluster_7', 'cluster_8', 'cluster_9',
       'cluster_10', 'cluster_11', 'cluster_12', 'cluster_13',
       'cluster_14', 'cluster_15', 'cluster_16'], dtype=object)

In [30]:
holidays.type.unique()

array([0, 4, 2, 5, 1, 3], dtype=int64)

2. The dayofyear column in our dates dataset ranges from 1 to 366. This will make some days fall on the wrong number 

Solution: find the sine and cosine of this column to represent the cyclic nature of a year. We can also include weather conditions, holidays and events to this.

In [31]:
dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1320 entries, 0 to 1319
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   date              1320 non-null   int64
 1   year              1320 non-null   int64
 2   month             1320 non-null   int64
 3   dayofmonth        1320 non-null   int64
 4   dayofweek         1320 non-null   int64
 5   dayofyear         1320 non-null   int64
 6   weekofyear        1320 non-null   int64
 7   quarter           1320 non-null   int64
 8   is_month_start    1320 non-null   bool 
 9   is_month_end      1320 non-null   bool 
 10  is_quarter_start  1320 non-null   bool 
 11  is_quarter_end    1320 non-null   bool 
 12  is_year_start     1320 non-null   bool 
 13  is_year_end       1320 non-null   bool 
 14  year_weekofyear   1320 non-null   int64
dtypes: bool(6), int64(9)
memory usage: 100.7 KB


In [32]:
# create new coolumns to represent the cyclic nature of a year
dates["sin(dayofyear)"] = np.sin(dates["dayofyear"])
dates["cos(dayofyear)"] = np.cos(dates["dayofyear"])

In [33]:
def get_datetime(df):
  # Create a new column combining the year, month, and day of the month in the desired format
  df['date_extracted'] = (
      dates['year'].astype(int).add(2000).astype(str) + '-' +
      dates['month'].astype(str).str.zfill(2) + '-' +
      dates['dayofmonth'].astype(str).str.zfill(2)
  )

get_datetime(dates)

### merging our data

In [34]:
stores.rename(  # rename type to store_type to make it more descriptive
      columns={'type': 'store_type'}, 
      inplace=True) 
holidays.rename(  # rename type to holiday_type to make it more descriptive
      columns={'type': 'holiday_type'}, 
      inplace=True)
# make each holiday type a string
holidays['holiday_type'] = holidays['holiday_type'].apply(lambda x: 'holiday_' + str(x))

In [35]:
#merging train and test with stores dataset

def merge(df1, df2): 
    merged_df = pd.merge( #left join with stores as 2nd df
      df1,
      df2,
      how='left', 
      on=['date'])

    return merged_df

def merge_stores(df1, df2): 
    merged_df = pd.merge( #left join with stores as 2nd df
      df1,
      df2,
      how='left', 
      on=['store_id'])

    return merged_df

In [36]:
def get_is_holiday_column(df):
  df['holiday_type'] = df['holiday_type'].fillna('Workday')

  # create column to show if its a holiday or not (non-holidays are zeros)
  df['is_holiday'] = df['holiday_type'].apply(
      lambda x: False if x=='Workday'
      else True)

we did this so our non-holidays can be zeros

now we must merge holidays with the merged data

since non-holidays are zeros, we don't want our ML Models to think that non-holidays(zeros) have an ordinal relationship with other holidays(1,2,3,4,) 

in other words, non-holidays(zeros) don't always come before holidays(1,2,3,4,)

so, we must create a new column to show whether or not.

In [37]:
train_merged = merge_stores(train, stores)
train_merged1 = merge(train_merged, holidays)
get_is_holiday_column(train_merged1)
train_merged2 = merge(train_merged1, dates)

test_merged = merge_stores(test, stores)
test_merged1 = merge(test_merged, holidays)
get_is_holiday_column(test_merged1)
test_merged2 = merge(test_merged1, dates)

In [38]:
train_merged2['holiday_type'].unique()

array(['holiday_0', 'Workday', 'holiday_2', 'holiday_5', 'holiday_1',
       'holiday_4', 'holiday_3'], dtype=object)

In [46]:
def set_index(df):
  df.drop('date', inplace=True, axis=1)
  df.set_index('date_extracted', inplace=True)
set_index(train_merged2)
set_index(test_merged2)

## Drop Duplicates

In [50]:
train_merged2.drop_duplicates(inplace=True)
test_merged2.drop_duplicates(inplace=True)

In [51]:
train = train_merged2
test = test_merged2

## Impute Missing Values

In [52]:
print(train.isnull().sum())
print(test.isnull().sum())

store_id               0
category_id            0
target                 0
onpromotion            0
nbr_of_transactions    0
city                   0
store_type             0
cluster                0
holiday_type           0
is_holiday             0
year                   0
month                  0
dayofmonth             0
dayofweek              0
dayofyear              0
weekofyear             0
quarter                0
is_month_start         0
is_month_end           0
is_quarter_start       0
is_quarter_end         0
is_year_start          0
is_year_end            0
year_weekofyear        0
sin(dayofyear)         0
cos(dayofyear)         0
dtype: int64
store_id            0
category_id         0
onpromotion         0
city                0
store_type          0
cluster             0
holiday_type        0
is_holiday          0
year                0
month               0
dayofmonth          0
dayofweek           0
dayofyear           0
weekofyear          0
quarter             0
is_mont

# Exploratory Data Analysis: EDA

## Hypothesis Validation
**H0**: holidays have a big effect on sales, hence the sales data is seasonal.

**H1**: holidays don't affect sales, hence sales data is stationary.

In [53]:
train.tail()

,store_id,category_id,target,onpromotion,nbr_of_transactions,city,store_type,cluster,holiday_type,is_holiday,...,quarter,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,sin(dayofyear),cos(dayofyear)
date_extracted,,,,,,,,,,,,,,,,,,,,,
2004-06-18,store_9,category_23,539.725,0,2141.0,city_0,store_1,cluster_4,Workday,False,...,2,False,False,False,False,False,False,424,-0.602,0.798496
2004-06-18,store_9,category_20,84.177,0,2141.0,city_0,store_1,cluster_4,Workday,False,...,2,False,False,False,False,False,False,424,-0.602,0.798496
2004-06-18,store_9,category_15,1973.760,3,2141.0,city_0,store_1,cluster_4,Workday,False,...,2,False,False,False,False,False,False,424,-0.602,0.798496
2004-06-18,store_9,category_29,2.000,0,2141.0,city_0,store_1,cluster_4,Workday,False,...,2,False,False,False,False,False,False,424,-0.602,0.798496
2004-06-18,store_9,category_10,27.076,0,2141.0,city_0,store_1,cluster_4,Workday,False,...,2,False,False,False,False,False,False,424,-0.602,0.798496


In [54]:
train.columns

Index(['store_id', 'category_id', 'target', 'onpromotion',
       'nbr_of_transactions', 'city', 'store_type', 'cluster', 'holiday_type',
       'is_holiday', 'year', 'month', 'dayofmonth', 'dayofweek', 'dayofyear',
       'weekofyear', 'quarter', 'is_month_start', 'is_month_end',
       'is_quarter_start', 'is_quarter_end', 'is_year_start', 'is_year_end',
       'year_weekofyear', 'sin(dayofyear)', 'cos(dayofyear)'],
      dtype='object')

## Answering Questions

1. Is the train data complete?

Yes. The output below shows that our train data is incomplete.

In [55]:
# create a function to check for missing extracted dates
def get_missing_dates(df):
  col = df.index
  missing_dates = (pd.date_range(
      
      start=col.min(), #start date
      end=col.max())   #end_date
      .difference(col))
  print(f"we have {len(missing_dates)} dates missing out of {len(col)}")
  return missing_dates

In [56]:
get_missing_dates(train)

we have 1265 dates missing out of 2268486


DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10',
               ...
               '2004-06-09', '2004-06-10', '2004-06-11', '2004-06-12',
               '2004-06-13', '2004-06-14', '2004-06-15', '2004-06-16',
               '2004-06-17', '2004-06-18'],
              dtype='datetime64[ns]', length=1265, freq=None)

2. Do we have seasonality in our sales?

In [57]:
# Assuming your time series data is stored in the variable 'sales_data'
sales_data = train['target']

In [58]:
# Perform KPSS test
kpss_result = kpss(sales_data)
kpss_statistic = kpss_result[0]
kpss_pvalue = kpss_result[1]
kpss_critical_values = kpss_result[3]

In [59]:
print("\nKPSS Test:")
print("KPSS Statistic:", kpss_statistic)
print("p-value:", kpss_pvalue)


KPSS Test:
KPSS Statistic: 64.85234409929284
p-value: 0.01


stationary if p-value > 0.05

series is stationary since 0.01 < 0.05

In [64]:
def check_stationarity(df, date_col, target_col, window=12):
    # Calculate rolling statistics
    rolling_std = df[target_col].rolling(window=window).std()
    rolling_mean = df[target_col].rolling(window=window).mean()

    # Plot original series and rolling statistics
    plt.figure(figsize=(10, 6))
    plt.plot(df.index, df[target_col], color='blue', label='Original Series')
    plt.plot(df.index, rolling_std, color='green', label='Rolling Std')
    plt.plot(df.index, rolling_mean, color='red', label='Rolling Mean')
    plt.legend()
    plt.title('Rolling Statistics')
    plt.xlabel('Date')
    plt.ylabel('Target(sales)')
    plt.tight_layout()  # Adjusts plot spacing
    plt.show()

# Example usage
df = sales_data  # Assuming the sales data is stored in a dataframe called sales_data
target_col = 'sales'  # Column containing the sales data

check_stationarity(train, 'date_extracted', 'target')

series is stationary from the above plot, since the statistic(standard deviation) remains similar to the original series

In [61]:
# pacf = plot_pacf(series,lags = 10)

3. Are there outliers in our dataset?

4. What is the difference between RMSLE, RMSE and MSE?

## Univariate Analysis

## Bivariate Analysis

## Multivariate Analysis

# Feature Engineering

## Creating New Features

In [ ]:
def getDateFeatures(df):
    
    df["is_weekend"] = np.where(df['dayofweek'] > 4, 1, 0)

    # Define the criteria for each season
    seasons = {'Winter': [12, 1, 2], 'Spring': [3, 4, 5], 'Summer': [6, 7, 8], 'Autumn': [9, 10, 11]}

    # Create the 'season' column based on the 'date' column
    df['season'] = df["month"].map({month: season for season, months in seasons.items() for month in months})

    return df

In [ ]:
getDateFeatures(train)
getDateFeatures(test)

,store_id,category_id,onpromotion,date_extracted,city,store_type,cluster,holiday_type,is_holiday,year,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,sin(dayofyear),cos(dayofyear),is_weekend,season
0,store_1,category_24,0.0,2004-06-19,city_0,store_0,cluster_0,0.0,False,4,...,False,False,False,False,False,425,0.346649,0.937995,0,Summer
1,store_1,category_21,0.0,2004-06-19,city_0,store_0,cluster_0,0.0,False,4,...,False,False,False,False,False,425,0.346649,0.937995,0,Summer
2,store_1,category_32,0.0,2004-06-19,city_0,store_0,cluster_0,0.0,False,4,...,False,False,False,False,False,425,0.346649,0.937995,0,Summer
3,store_1,category_18,16.0,2004-06-19,city_0,store_0,cluster_0,0.0,False,4,...,False,False,False,False,False,425,0.346649,0.937995,0,Summer
4,store_1,category_26,0.0,2004-06-19,city_0,store_0,cluster_0,0.0,False,4,...,False,False,False,False,False,425,0.346649,0.937995,0,Summer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210265,store_9,category_8,0.0,2004-08-13,city_0,store_1,cluster_4,0.0,False,4,...,False,False,False,False,False,432,-0.930095,0.367319,1,Summer
210268,store_9,category_13,0.0,2004-08-13,city_0,store_1,cluster_4,0.0,False,4,...,False,False,False,False,False,432,-0.930095,0.367319,1,Summer
210272,store_9,category_20,0.0,2004-08-13,city_0,store_1,cluster_4,0.0,False,4,...,False,False,False,False,False,432,-0.930095,0.367319,1,Summer
210273,store_9,category_15,0.0,2004-08-13,city_0,store_1,cluster_4,0.0,False,4,...,False,False,False,False,False,432,-0.930095,0.367319,1,Summer


let's make the date column the index

In [ ]:
train

,store_id,category_id,target,onpromotion,nbr_of_transactions,city,store_type,cluster,holiday_type,is_holiday,...,is_month_end,is_quarter_start,is_quarter_end,is_year_start,is_year_end,year_weekofyear,sin(dayofyear),cos(dayofyear),is_weekend,season
date,,,,,,,,,,,,,,,,,,,,,
2001-01-01,store_1,category_24,0.0,0.0,0.0,city_0,store_0,cluster_0,1.0,True,...,False,True,False,True,False,101.0,0.841471,0.540302,0,Winter
2001-01-01,store_1,category_21,0.0,0.0,0.0,city_0,store_0,cluster_0,1.0,True,...,False,True,False,True,False,101.0,0.841471,0.540302,0,Winter
2001-01-01,store_1,category_32,0.0,0.0,0.0,city_0,store_0,cluster_0,1.0,True,...,False,True,False,True,False,101.0,0.841471,0.540302,0,Winter
2001-01-01,store_1,category_18,0.0,0.0,0.0,city_0,store_0,cluster_0,1.0,True,...,False,True,False,True,False,101.0,0.841471,0.540302,0,Winter
2001-01-01,store_1,category_26,0.0,0.0,0.0,city_0,store_0,cluster_0,1.0,True,...,False,True,False,True,False,101.0,0.841471,0.540302,0,Winter
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004-06-18,store_9,category_23,0.0,0.0,0.0,city_0,store_1,cluster_4,0.0,False,...,False,False,False,False,False,424.0,-0.602000,0.798496,1,Summer
2004-06-18,store_9,category_20,0.0,0.0,0.0,city_0,store_1,cluster_4,0.0,False,...,False,False,False,False,False,424.0,-0.602000,0.798496,1,Summer
2004-06-18,store_9,category_15,0.0,0.0,0.0,city_0,store_1,cluster_4,0.0,False,...,False,False,False,False,False,424.0,-0.602000,0.798496,1,Summer


## Resampling

In [ ]:
weekly_df = train.resample('W').sum()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'

## Features Encoding & scaling

In [ ]:
numeric_columns = train.select_dtypes(include=['int64', 'float64']).columns.tolist()
categoric_columns = train.select_dtypes(include=['object', 'category']).columns.tolist()

In [ ]:
numeric_columns.remove('sales')
numeric_columns.remove('nbr_of_transactions')
print(numeric_columns)

ValueError: list.remove(x): x not in list

In [ ]:
encoder = BinaryEncoder(drop_invariant=False, return_df=True,)
encoder.fit(train[categoric_columns])

MemoryError: Unable to allocate 214. MiB for an array with shape (4668381, 6) and data type int64

In [ ]:
scaler = StandardScaler()
scaler.set_output(transform="pandas")
scaler.fit(train[numeric_columns])

StandardScaler()

In [ ]:
# import pickle

# with open('encoder.pkl', 'wb') as f:
#     pickle.dump(encoder, f)

# with open('scaler.pkl', 'wb') as f:
#     pickle.dump(scaler, f)

In [ ]:
scaled_num = scaler.transform(train[numeric_columns])
scaled_num_test = scaler.transform(test[numeric_columns])

In [ ]:
encoded_cat = encoder.transform(merged3[categoric_columns])
encoded_cat_test = encoder.transform(merged3_test[categoric_columns])

In [ ]:
merged3 = pd.concat([scaled_num, encoded_cat], axis=1)
merged3_test = pd.concat([scaled_num_test, encoded_cat_test], axis=1)

In [ ]:
# sales_data = train_daily_mean.drop(columns=['store_nbr', 'onpromotion'])
sales_data = train_daily_mean['sales']
sales_data = sales_data.fillna(0)

In [ ]:
train1=train_daily_mean

In [ ]:
trainn, evall = merged3[30000:], merged3[:30000]

In [ ]:
# Split data into parts
x = trainn.drop(['sales'], axis = 1)
y = trainn['sales']

In [ ]:
# Split data into Train Test
X_train, X_test, y_train, y_test = x[200:], x[:200], y[200:], y[:200]

# Machine Learning Modeling 

# Traditional Machine Learning Models

### DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree = DecisionTreeRegressor()
model_tree = tree.fit(X_train, y_train)

# Make prediction on X_test
tree_pred = model_tree.predict(X_test)

In [ ]:
with open("model.pkl", "wb") as f:
    pickle.dump(model_tree, f)

In [ ]:
# feature importance for decision tree
plt.figure(figsize=(12,7))
plt.barh(x.columns, model_tree.feature_importances_)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(y_test, label ='Actual Sales')
plt.plot(tree_pred, label='DecisionTreeRegressor')
plt.legend(loc='best')
plt.title('DecisionTreeRegressor Prediction')
plt.show()

In [ ]:
mse = mean_squared_error(y_test, tree_pred )
rmse = np.sqrt(mean_squared_error(y_test, tree_pred )).round(2)
rmsle = np.sqrt(mean_squared_log_error(y_test, tree_pred)).round(2)
msle = mean_squared_log_error(y_test, tree_pred).round(2)


results = pd.DataFrame([['DecisionTree', mse, msle, rmse, rmsle]], columns = ['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])
# results = results.append(model_results, ignore_index = True)
results

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
neigh = KNeighborsRegressor(n_neighbors=1)
# fit model no training data
neigh.fit(X_train, y_train)

# make predictions for test data
neigh_pred = neigh.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, neigh_pred )
msle = mean_squared_log_error(y_test, neigh_pred)
rmse = np.sqrt(mean_squared_error(y_test, neigh_pred )).round(2)
rmsle = np.sqrt(mean_squared_log_error(y_test, neigh_pred)).round(5)

# model_results = pd.DataFrame([['lightGBM', mse, rmse]], columns = ['Model', 'MSE', 'RMSE'])
model_results = pd.DataFrame([['KNN', mse, msle, rmse, rmsle]], columns = ['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])
results = results.append(model_results, ignore_index = True)
results

# Models Comparison

# Model Evaluation (Backtests)

In [ ]:
backtests = {
    1: ('2017-07-01', '2017-07-15'),
    2: ('2017-07-15', '2017-07-30'),
    3: ('2017-08-01', '2017-08-15')
}

In [ ]:
# Backtests with Decision Tree
scores = {}

for idx, period in enumerate(backtests):
    
    _train = trainn.reset_index()[trainn.reset_index()['date'] < backtests[period][0]]
    _test = trainn.reset_index()[(trainn.reset_index()['date'] >= backtests[period][0]) & (trainn.reset_index()['date'] <= backtests[period][1])]
    
    Xtrain, ytrain = _train.set_index(['date']).drop(columns=['sales']).values, _train.sales.values
    Xtest, ytest = _test.set_index(['date']).drop(columns=['sales']).values, _test.sales.values
    
    lgbm_model = DecisionTreeRegressor().fit(Xtrain, ytrain)
    
    ypred = lgbm_model.predict(Xtest)
    
    scores[period] = np.sqrt(mean_squared_log_error(ytest, ypred))

print(scores)

In [ ]:
# Split data into parts
x = evall.drop(['sales'], axis = 1)
y = evall['sales']

In [ ]:
# Make prediction on X_test
tree_pred = neigh.predict(x)

In [ ]:
mse = mean_squared_error(y, tree_pred )
rmse = np.sqrt(mean_squared_error(y, tree_pred )).round(2)
rmsle = np.sqrt(mean_squared_log_error(y, tree_pred)).round(6)
msle = mean_squared_log_error(y, tree_pred).round(2)


results = pd.DataFrame([['DecisionTree', mse, msle, rmse, rmsle]], columns = ['Model', 'MSE', 'MSLE', 'RMSE', 'RMSLE'])
results

### predicting sales in our test

In [ ]:
# test_pred = armodel.predict(merged3_test_resampled)
test_pred = armodel.predict(
    start=len(x),
    end=len(x)+ len(test_date) -1,
    dynamic=False
)
test_pred[np.isnan(test_pred)] = 0
test_pred

In [ ]:
test_sales = pd.DataFrame(test_pred, columns=['sales'])
test_sales.head(30)

In [ ]:
# Save sample submission
# test_sales[[ 'sales']].to_csv('submission.csv', index=False)